In [1]:
#|default_exp preprocessing
#|export

import polars as pl
import numpy as np

from datetime import datetime
import calendar

import json
import os

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

import torch
from torch.utils.data import DataLoader, TensorDataset 

In [2]:
#|export

class DataHandler():
    def __init__(self, csv_path: str = '../local_data/all_lending_club_loan_data_2007-2018.csv'):
        self.cleaned_csv_path = f'{csv_path[:-4]}_cleaned.csv'
        self.features_path = f'{csv_path[:-4]}_cleaned_features.json'

        if not os.path.exists(self.cleaned_csv_path):
            self.strip_non_data_rows_from_lending_club_data(csv_path)
            self.clean_lending_club_data()

        with open(self.features_path, 'r') as f:
            self.features = json.load(f)

        self.transformer = self.column_transformer = ColumnTransformer(
            transformers=[
                ('standard_scaler', StandardScaler(), self.features['standard_scaler']),
                ('min_max_scaler', MinMaxScaler(), self.features['min_max_scaler']),
                ('one_hot_encoder', OneHotEncoder(drop=None, handle_unknown='ignore'), self.features['categorical'] )
            ],
        )

        self.training_data_start_date = None
        self.training_data_end_date = None
    
    def strip_non_data_rows_from_lending_club_data(self, csv_path: str):
        with open(csv_path, 'r') as f:
            lines = [line for line in f if line[0].isdigit() or line.startswith('id')]

        with open('temp.csv', 'w') as f:
            f.writelines(lines)

    def is_date_value(self, value: str) -> bool:
        '''
        Check if a value matches month-year format (e.g., 'sep-2015')
        '''
        if not isinstance(value, str) or '-' not in value:
            return False
        
        try:
            month_str, year_str = value.lower().split('-')
            return (month_str.title() in calendar.month_abbr and 
                    year_str.isdigit() and 
                    len(year_str) == 4)
        except:
            return False
        
    def parse_date_value(self, value: str) -> datetime:
            '''
            Convert month-year string to datetime
            '''
            if not isinstance(value, str):
                return None
            
            try:
                month_str, year_str = value.lower().split('-')
                month_num = list(calendar.month_abbr).index(month_str.title())
                return datetime(int(year_str), month_num, 1)
            except:
                return None
            
    def drop_undesired_columns(self, df: pl.DataFrame)-> pl.DataFrame:
        explicit_columns_to_drop = [
            'id',                       # Unique identifier
            'funded_amnt',              # Redundant due to loan_amnt
            'funded_amnt_inv',          # Redundant due to loan_amnt
            'sub_grade',                # Redundant due to grade column
            'emp_title',                # Too random
            'title',                    # Redundant in relation to purpose column
            'desc',                     # Mostly null
            'url',                      # No predictive value
            'mths_since_last_delinq',   # Mostly null and redundant due to delinq_2yrs column
            'mths_since_last_record',   # Mostly null
            'pymnt_plan',               # Always 'n'
            'addr_state',               # Reduce dimensionality of this excerise
            'zip_code'                  # Reduce dimensionality of this excerise
        ]

        # Dropping all secondary applicant, hardship, settlement, and joint columns as they are all null
        implicit_columns_to_drop = (
            'sec_app_',
            'hardship_',
            'settlement_',
            'joint_',
        )

        implicit_columns_to_drop = [column for column in df.columns if column.startswith(implicit_columns_to_drop)]
        high_null_cols = [column for column in df.columns  if (df[column].null_count() / len(df)) > 0.2]

        columns_to_drop = explicit_columns_to_drop + implicit_columns_to_drop + high_null_cols
        columns_to_drop = list(set(columns_to_drop))
        
        return df.drop(columns_to_drop)

    def convert_employment_length(self, value: str) -> float:
        if value is None or value == 'n/a':
            return None
        if value == '< 1 year':
            return 0.5
        if value == '10+ years':
            return 10.0
        return float(value.split()[0])

    def clean_lending_club_data(self):
        '''
        Process Lending Club data:
        1. Identify null and date (month-year format) columns
        2. Deelete null columns
        2. Convert identified date columns to datetime
        '''

        df = pl.read_csv('temp.csv')

        null_columns = []
        date_columns = []
        
        #Identify null and date columns
        for col in df.columns:
            # Check if column is all null
            if df[col].is_null().all():
                null_columns.append(col)
                continue
            
            #Identify date columns
            sample = df[col].drop_nulls().sample(1)
            if self.is_date_value(str(sample[0])):
                date_columns.append(col)

        df = df.drop(null_columns)

        #Convert date columns to datetime
        if date_columns:
            df = df.with_columns([
                pl.col(col)
                .str.to_lowercase()
                .map_elements(self.parse_date_value, return_dtype=datetime)
                .alias(col)
                for col in date_columns
            ])

            #Include month and year columns for each date column
            expressions = []
            for col in date_columns:
                expressions.extend([
                    (pl.col(col).cast(pl.Int64)/pl.lit(2.628e+15)).alias(f'{col}_unicode_month')
                ])
            df = df.with_columns(expressions)

        df = df.with_columns([
            pl.col('term').str.extract(r'(\d+)').cast(pl.Int64).alias('term_months'),
            pl.col('emp_length').map_elements(self.convert_employment_length, return_dtype=pl.Float64).alias('employment_years'),
            pl.col('grade').str.to_uppercase().map_elements(lambda x: ord(x) - 64, return_dtype=pl.Int64).alias('grade'),
            pl.col('debt_settlement_flag').map_elements(lambda x: 1 if x == 'Y' else 0, return_dtype=pl.Int64).alias('debt_settlement_flag'),
            pl.col('orig_projected_additional_accrued_interest').cast(pl.Float64).alias('orig_projected_additional_accrued_interest')
            ]).drop(['emp_length', 'term'])
        
        df = self.drop_undesired_columns(df)

        os.remove('temp.csv')

        standard_scaler_columns = [column for column, dtype in df.schema.items() 
                           if dtype==pl.Float64]
        min_max_scaler_columns = [column for column, dtype in df.schema.items() 
                           if dtype==pl.Int64]
        categorical_columns = [col for col, dtype in df.schema.items() 
                               if dtype == pl.Utf8]

        features_dict = {
            'standard_scaler': standard_scaler_columns,
            'min_max_scaler': min_max_scaler_columns,
            'categorical': categorical_columns
        }

        with open(self.features_path, 'w') as f:
            json.dump(features_dict, f, indent=2)

        df = df.with_columns([
            pl.col(col).fill_null('missing') for col in categorical_columns
        ])

        df.write_csv(self.cleaned_csv_path)

    def get_data_by_date_range(self, start_date: datetime, end_date: datetime, date_column: str = 'issue_d', return_unlisted_columns: bool = False):
        '''
        Extract rows between two datetime values using a lazy frame
        '''
        lf = pl.scan_csv(
            self.cleaned_csv_path, 
            low_memory=True,
            try_parse_dates=True)
        
        filtered_lf = lf.filter(
            pl.col(date_column).is_between(start_date, end_date)
        )

        if return_unlisted_columns:
            return filtered_lf.collect()
        else:
            df = filtered_lf.collect()
            df = filtered_lf.collect()

            datetime_columns = [column for column in df.columns if df[column].dtype == pl.Datetime]

            return df.drop(datetime_columns)

    def get_train_data(self, start_date: datetime, end_date: datetime)-> tuple[np.ndarray, np.ndarray]:
        '''Get transformed data ready for the autoencoder'''
        self.training_data_start_date = start_date
        self.training_data_end_date = end_date

        raw_data = self.get_data_by_date_range(start_date, end_date, date_column='issue_d', return_unlisted_columns=False)
        transformed_data = self.transformer.fit_transform(raw_data)
        
        missing_mask = ~np.isnan(transformed_data)
        transformed_data[~missing_mask] = 0.0

        return transformed_data, missing_mask

    def get_test_data(self, start_date: datetime, end_date: datetime)-> tuple[np.ndarray, np.ndarray]:
        if self.training_data_start_date is None or self.training_data_end_date is None:
            raise ValueError('Training data not set. Please call get_training_data first.')
        
        if end_date < self.training_data_start_date or start_date > self.training_data_end_date:
            raw_data = self.get_data_by_date_range(start_date, end_date, date_column='issue_d', return_unlisted_columns=False)
            transformed_data = self.transformer.transform(raw_data)
        
            not_null_mask = ~np.isnan(transformed_data)
            transformed_data[~not_null_mask] = 0.0

            return transformed_data, not_null_mask
            
        else:
            raise ValueError('There is an overlap between the training and test data.')
        
    def get_transformed_data_feature_names(self):
        return self.transformer.get_feature_names_out()
    
    def get_sigmoid_feature_mask(self, as_torch=False):
        mask = np.array([1 if ('one_hot_encoder' in name or 'min_max_scaler' in name) else 0
                         for name in self.transformer.get_feature_names_out()])
        
        if as_torch:
            return torch.tensor(mask, dtype=torch.bool)
        else:
            return mask
    
    def get_binary_feature_mask(self, as_torch=False):
        mask = np.array([1 if ('one_hot_encoder' in name or 'flag' in name) else 0
                         for name in self.transformer.get_feature_names_out()])
        if as_torch:
            return torch.tensor(mask,dtype=torch.bool)
        else:
            return mask

In [18]:
#|export

def to_torch_dataloader(data, not_null_mask, batch_size: int = 64):
    data_tensor = torch.FloatTensor(data)
    not_null_mask_tensor = torch.BoolTensor(not_null_mask)

    # Create a dataset
    dataset = TensorDataset(data_tensor, not_null_mask_tensor)

    # Create dataloaders
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
#|test

data_handler = DataHandler(csv_path='../local_data/all_lending_club_loan_data_2007-2018.csv')

start = datetime(2010,1,1)
end = datetime(2010,12,31)
train_data, train_mask = data_handler.get_train_data(start, end)

start = datetime(2011,1,1)
end = datetime(2011,12,31)
test_data, test_mask = data_handler.get_test_data(start, end)

if (train_data.shape[1] != test_data.shape[1]):
    raise ValueError('Training and testing data have different numbers of features')

#Make into torch dataloader
data_loader = to_torch_dataloader(train_data, train_mask)

if type(data_loader) != DataLoader:
    raise ValueError('Data loader is not a DataLoader')



In [5]:
data_handler = DataHandler(csv_path='../local_data/all_lending_club_loan_data_2007-2018.csv')

start = datetime(2017,1,1)
end = datetime(2018,12,30)
data = data_handler.get_data_by_date_range(start, end)

display(data)

loan_amnt,int_rate,installment,grade,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,…,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,disbursement_method,debt_settlement_flag,issue_d_unicode_month,earliest_cr_line_unicode_month,last_pymnt_d_unicode_month,last_credit_pull_d_unicode_month,term_months,employment_years
f64,f64,f64,i64,str,f64,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,i64,f64,f64,f64,f64,i64,f64
5000.0,20.39,186.82,4,"""RENT""",50000.0,"""Verified""","""Current""","""other""",21.8,1.0,665.0,669.0,0.0,5.0,0.0,116.0,23.2,18.0,"""w""",3780.31,3780.31,2043.69,2043.69,1219.69,824.0,0.0,0.0,0.0,186.82,609.0,605.0,0.0,1.0,"""Individual""",0.0,0.0,…,80.0,13.0,11.0,11.0,0.0,11.0,2.0,4.0,2.0,2.0,2.0,2.0,15.0,2.0,2.0,2.0,5.0,0.0,0.0,1.0,1.0,77.8,0.0,0.0,0.0,33430.0,19344.0,500.0,27820.0,"""Cash""",0,0.578334,0.468329,0.590334,0.590334,36,8.0
15000.0,9.92,483.45,2,"""OWN""",196000.0,"""Source Verified""","""Current""","""debt_consolidation""",18.29,0.0,700.0,704.0,0.0,19.0,0.0,24243.0,46.3,53.0,"""w""",10878.5,10878.5,5301.42,5301.42,4121.5,1179.92,0.0,0.0,0.0,483.45,694.0,690.0,0.0,1.0,"""Individual""",0.0,0.0,…,141.0,236.0,4.0,4.0,5.0,11.0,6.0,0.0,4.0,10.0,5.0,16.0,11.0,14.0,37.0,10.0,19.0,0.0,0.0,0.0,6.0,98.0,75.0,0.0,0.0,605228.0,137713.0,28500.0,147178.0,"""Cash""",0,0.578334,0.342181,0.589414,0.590334,36,2.0
11200.0,30.79,367.82,7,"""RENT""",44000.0,"""Not Verified""","""Current""","""medical""",43.97,1.0,665.0,669.0,2.0,8.0,0.0,1526.0,24.6,14.0,"""w""",10193.73,10193.73,4007.7,4007.7,1006.27,3001.43,0.0,0.0,0.0,367.82,629.0,625.0,0.0,1.0,"""Joint App""",0.0,0.0,…,124.0,128.0,5.0,5.0,0.0,34.0,0.0,1.0,2.0,3.0,2.0,3.0,8.0,4.0,6.0,3.0,8.0,0.0,0.0,0.0,2.0,71.4,0.0,0.0,0.0,80367.0,67173.0,1900.0,74167.0,"""Cash""",0,0.578334,0.450247,0.589414,0.590334,60,0.5
25000.0,21.85,688.35,4,"""MORTGAGE""",65000.0,"""Source Verified""","""Current""","""debt_consolidation""",12.89,1.0,665.0,669.0,1.0,7.0,0.0,8657.0,98.4,16.0,"""w""",22188.73,22188.73,7511.16,7511.16,2811.27,4699.89,0.0,0.0,0.0,688.35,669.0,665.0,0.0,1.0,"""Individual""",0.0,0.0,…,69.0,126.0,72.0,16.0,2.0,126.0,0.0,2.0,1.0,3.0,1.0,1.0,4.0,3.0,9.0,3.0,7.0,0.0,0.0,1.0,0.0,75.0,100.0,0.0,0.0,101234.0,17039.0,3300.0,10220.0,"""Cash""",0,0.578334,0.302137,0.589414,0.590334,60,10.0
3000.0,7.34,93.1,1,"""RENT""",52000.0,"""Source Verified""","""Fully Paid""","""major_purchase""",0.58,0.0,760.0,764.0,0.0,7.0,0.0,141.0,0.5,30.0,"""w""",0.0,0.0,3011.577285,3011.58,3000.0,11.58,0.0,0.0,0.0,614.03,764.0,760.0,0.0,1.0,"""Individual""",0.0,0.0,…,132.0,242.0,18.0,7.0,4.0,18.0,7.0,0.0,1.0,1.0,4.0,15.0,7.0,6.0,19.0,1.0,7.0,0.0,0.0,0.0,1.0,96.7,0.0,0.0,0.0,191216.0,141.0,30500.0,0.0,"""Cash""",0,0.578334,0.33623,0.58034,0.586389,36,9.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
12000.0,14.08,279.72,3,"""MORTGAGE""",58000.0,"""Not Verified""","""